In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
batch_size = 1970
n_batch = mnist.train.num_examples // batch_size

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    #x: input tensor of shape [batch, in_height, in_width, in_channels]
    #W filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #strides[0] = strides[3] = 1, strides[1]: step width on x direction, strides[2]: step width on y direction
    #Padding: a string, valid values are: "SAME" and "VALID"
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool2x2(x):
    #kernel size [1, x, y, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')




In [4]:
with tf.device('/gpu:0'):
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, 784]) #784=28*28
        y = tf.placeholder(tf.float32, [None, 10])

    with tf.name_scope('reshape_input'):
        #change the format of x to shape [batch, in_height, in_width, in_channels]
        x_image = tf.reshape(x, [-1,28,28,1])

    with tf.name_scope('conv_layer1'):
        with tf.name_scope('weight_conv1'):
            #initialize weights and biases for convolution layer 1
            w_conv1 = weight_variable([5,5,1,32]) #sampling window of 5*5, convolution is done on 1 plane with 32 kernels
        with tf.name_scope('bias_conv1'):
            b_conv1 = bias_variable([32])

        with tf.name_scope('relu_layer1'):
            #do convolution on x_image with weight and bias, using relu activation algorithm, then do max_pooling
            h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
        with tf.name_scope('pooling_layer1'):
            h_pool1 = max_pool2x2(h_conv1)

    with tf.name_scope('conv_layer2'):
        #initialize weights and biases for convolution layer 2
        with tf.name_scope('weight_conv2'):
            w_conv2 = weight_variable([5,5,32,64]) #sampling window of 5*5, convolution is done on 32 planes with 64 kernels
        with tf.name_scope('bias_conv2'):
            b_conv2 = bias_variable([64])

        with tf.name_scope('relu_layer2'):
            #do convolution on x_image with weight and bias, using relu activation algorithm, then do max_pooling
            h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
        with tf.name_scope('pooling_layer2'):
            h_pool2 = max_pool2x2(h_conv2)

    #images of 28*28 is still 28*28 after the first convoluton, and it becomes 14*14 after the first pooling
    #images of 14*14 is still 14*14 after the 2nd convoluton, and it becomes 7*7 after the 2nd pooling
    #we'd get 64 planes of size 7*7

    with tf.name_scope('full_connect1'):
        with tf.name_scope('w_fc1'):
            #ouput of last layer is 7*7*64 neural units, and we set 1024 neural units for the 1st fully connect layer
            w_fc1 = weight_variable([7*7*64,1024])
        with tf.name_scope('bias_fc1'):
            b_fc1 = bias_variable([1024])

    with tf.name_scope('flaten_the_2nd_pooling_layer'):
        #flaten the output of the 2nd pooling layer
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

    with tf.name_scope('relu_fc1'):
        #get the output of the 1st fc layer
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

    with tf.name_scope('drop_out_op'):
        with tf.name_scope('keep_prob'):
            #keep_prob is the probability of output from a neural layer
            keep_prob = tf.placeholder(tf.float32)
        with tf.name_scope('drop_out'):
            h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    with tf.name_scope('full_connect2'):
        with tf.name_scope('w_fc2'):
            #the 2nd fc layer
            w_fc2 = weight_variable([1024,10])
        with tf.name_scope('b_fc2'):
            b_fc2 = bias_variable([10])

    with tf.name_scope('softmax'):
        #calculate the output
        prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

    with tf.name_scope('reduce_mean'):
        #the cross entropy cost function
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))

    with tf.name_scope('AdamOptimizer'):
        #optimize with AdamOptimizer
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    with tf.name_scope('calculate_prediction'):
        with tf.name_scope('prediction'):
            #the result saves in a bool list
            correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        with tf.name_scope('calculate_accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            with tf.device('/cpu:0'):
                tf.summary.scalar('accuracy', accuracy)

    with tf.name_scope('init'):
        init = tf.global_variables_initializer()

with tf.device('/cpu:0'):
    merged = tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [5]:
with tf.name_scope('training'):
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=True)) as sess:
        sess.run(init)
        writer = tf.summary.FileWriter('logs/', sess.graph)
        for epoch in range (21):
            for batch in range(n_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                summary, _ = sess.run([merged, train_step], feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.7})

            writer.add_summary(summary, epoch)
            acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
            
            print("Iter: {}, Test Accuracy: {}".format(epoch, acc))

Iter: 0, Test Accuracy: 0.5224000215530396
Iter: 1, Test Accuracy: 0.664900004863739
Iter: 2, Test Accuracy: 0.8312000036239624
Iter: 3, Test Accuracy: 0.8485000133514404
Iter: 4, Test Accuracy: 0.8561000227928162
Iter: 5, Test Accuracy: 0.8623999953269958
Iter: 6, Test Accuracy: 0.8664000034332275
Iter: 7, Test Accuracy: 0.8700000047683716
Iter: 8, Test Accuracy: 0.8730000257492065
Iter: 9, Test Accuracy: 0.8751999735832214
Iter: 10, Test Accuracy: 0.8758000135421753
Iter: 11, Test Accuracy: 0.8766999840736389
Iter: 12, Test Accuracy: 0.8783000111579895
Iter: 13, Test Accuracy: 0.8798999786376953
Iter: 14, Test Accuracy: 0.8808000087738037
Iter: 15, Test Accuracy: 0.8823999762535095
Iter: 16, Test Accuracy: 0.882099986076355
Iter: 17, Test Accuracy: 0.8830999732017517
Iter: 18, Test Accuracy: 0.9605000019073486
Iter: 19, Test Accuracy: 0.9700000286102295
Iter: 20, Test Accuracy: 0.9764000177383423
